First experiment with kaggle competition: Spaceship Titanic

https://www.kaggle.com/competitions/spaceship-titanic/overview

General Idea: Train a baseline model with Random Forest classifier.

Only do a quick EDA of the data and do just enough feature engineering to subit the model.

Result achieved on public leaderboard:


**0.79167**

In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [64]:
is_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', False)

In [65]:
if is_kaggle:
  # Input data files are available in the read-only "../input/" directory
  # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

  for dirname, _, filenames in os.walk('/kaggle/input'):
      for filename in filenames:
          print(os.path.join(dirname, filename))

  # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
  # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [66]:
base_path =  '/kaggle/input/spaceship-titanic' if is_kaggle else '/content/'

In [67]:
train_path = os.path.join(base_path, 'train.csv')
test_path = os.path.join(base_path, 'test.csv')

In [68]:
df = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [69]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [70]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [72]:
df["Transported"].value_counts()

,count
Transported,
True,4378
False,4315


First reasoning behind transormation of values is show. Then it is combined into single function

# Creation of new columns, because of combined information in fields

**Cabin, is in form: ( deck/num/side)**

Since second column is probably a number, so it makes sense to covert it to int.

In [73]:
df["Cabin"].str.split("/", expand=True)[1]

,1
0,0
1,0
2,0
3,0
4,1
...,...
8688,98
8689,1499
8690,1500
8691,608


We see that after conversion there are the same number of nulls, so yes all values are numbers

In [74]:
pd.to_numeric(df["Cabin"].str.split("/", expand=True)[1], errors='coerce').astype('Int64').isnull().sum()

np.int64(199)

In [75]:
df["Cabin"].str.split("/", expand=True)[1].isnull().sum()

np.int64(199)

Create the dictionaries

In [76]:
df[["CabinDeck", "CabinNum", "CabinSide"]] = df["Cabin"].str.split("/", expand=True)

In [77]:
df["CabinNum"] = df["CabinNum"].astype('Int64')

In [78]:
df["CabinDeck"].unique(), df["CabinDeck"].nunique()

(array(['B', 'F', 'A', 'G', nan, 'E', 'D', 'C', 'T'], dtype=object), 8)

In [79]:
df["CabinNum"].unique(), df["CabinNum"].nunique()

(<IntegerArray>
 [   0,    1,    2,    3, <NA>,    4,    5,    6,    8,    9,
  ...
  1782, 1884,  353, 1886, 1498, 1888, 1891, 1892, 1893, 1894]
 Length: 1818, dtype: Int64,
 1817)

In [80]:
df["CabinSide"].unique(), df["CabinSide"].nunique()

(array(['P', 'S', nan], dtype=object), 2)

**Passenger id is in form: gggg_pp, where gggg is group, pp is member of group**

In [81]:
df[['PassengerGroup', 'GroupMember']] = df['PassengerId'].str.split('_', expand=True)

# Check numerical Fields if Nan values are present

In [82]:
numeric_cols = df.select_dtypes(include=["number"]).columns
numeric_cols

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'CabinNum'],
      dtype='object')

In [83]:
df[numeric_cols].isnull().sum()

,0
Age,179
RoomService,181
FoodCourt,183
ShoppingMall,208
Spa,183
VRDeck,188
CabinNum,199


In [84]:
df[numeric_cols].describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CabinNum
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8494.0
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791,600.367671
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189,511.867226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,167.25
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,427.0
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000,999.0
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,1894.0


In [85]:
df[['Age','RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].median()

,0
Age,27.0
RoomService,0.0
FoodCourt,0.0
ShoppingMall,0.0
Spa,0.0
VRDeck,0.0


Since expect for age, other items are money spend on luxury, so they are present only for subset of customers.

Because of this zero (median) will be used on them.
For age average value is used instead.

In [86]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [87]:
cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df[cols] = df[cols].fillna(0)

Since 0 is already present as an acutal Cabin number, use -1 as Nan placholder.

In [88]:
df['CabinNum'] = df['CabinNum'].fillna(-1)

In [89]:
df[numeric_cols].isnull().sum()

,0
Age,0
RoomService,0
FoodCourt,0
ShoppingMall,0
Spa,0
VRDeck,0
CabinNum,0


# Categorical values

In [90]:
categorical_cols = df.select_dtypes(include=["object", "category"]).columns
categorical_cols

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP',
       'Name', 'CabinDeck', 'CabinSide', 'PassengerGroup', 'GroupMember'],
      dtype='object')

Name has mostly unique values, for now the column will be ignored

In [91]:
df['Name'].head()

,Name
0,Maham Ofracculy
1,Juanna Vines
2,Altark Susent
3,Solam Susent
4,Willy Santantines


In [92]:
len(df['Name'].unique()), len(df)

(8474, 8693)

Remove 2 variables splitted into columns from before and the Name column.

In [93]:
df.drop(columns=["Cabin", "PassengerId", "Name"], axis=1, inplace=True)

In [94]:
categorical_cols = df.select_dtypes(include=["object", "category"]).columns
categorical_cols

Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'CabinDeck',
       'CabinSide', 'PassengerGroup', 'GroupMember'],
      dtype='object')

In [95]:
df[categorical_cols].isnull().sum()

,0
HomePlanet,201
CryoSleep,217
Destination,182
VIP,203
CabinDeck,199
CabinSide,199
PassengerGroup,0
GroupMember,0


**1.) Categorical columns, that have basicaly true or valse value**

In [96]:
df["CryoSleep"].unique(), df["CryoSleep"].nunique()

(array([False, True, nan], dtype=object), 2)

In [97]:
df["VIP"].unique(), df["VIP"].nunique()

(array([False, True, nan], dtype=object), 2)

In [98]:
len(df)

8693

**Since for both fields, 1 is less frequent than 0, empty data will be filled with 0**

In [99]:
df["CryoSleep"].astype("Int64").value_counts(dropna=False)

,count
CryoSleep,
0,5439
1,3037
<NA>,217


In [100]:
df["VIP"].astype("Int64").value_counts(dropna=False)

,count
VIP,
0,8291
<NA>,203
1,199


**Convert the columns**

In [101]:
df["CryoSleep"] = df["CryoSleep"].astype("Int64").fillna(0)

In [102]:
df["VIP"] = df["VIP"].astype("Int64").fillna(0)

**2.) Categorical columns that have values, that are unordered, with couple of categories and also null values**

HomePlanet, CryoSleep,

CabinDeck, CabinSide   (generated from Cabin)

Have only small range of possible values,
they will be encoding categoricaly, with unkown for missing labels.

In [103]:
df["HomePlanet"].unique(), df["HomePlanet"].nunique()

(array(['Europa', 'Earth', 'Mars', nan], dtype=object), 3)

In [104]:
df["Destination"].unique(), df["Destination"].nunique()

(array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object), 3)

In [105]:
df["CabinDeck"].unique(), df["CabinDeck"].nunique()

(array(['B', 'F', 'A', 'G', nan, 'E', 'D', 'C', 'T'], dtype=object), 8)

In [106]:
df["CabinSide"].unique(), df["CabinSide"].nunique()

(array(['P', 'S', nan], dtype=object), 2)

Cabin number is probably a string representation of a number.

In [107]:
df["CabinNum"].isnull().sum()

np.int64(0)

In [108]:
pd.to_numeric(df["CabinNum"], downcast='integer')

,CabinNum
0,0
1,0
2,0
3,0
4,1
...,...
8688,98
8689,1499
8690,1500
8691,608


In [109]:
df = pd.get_dummies(df, columns= ['CabinDeck', 'CabinSide', 'HomePlanet', 'Destination'])

In [110]:
df

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,CabinNum,...,CabinDeck_G,CabinDeck_T,CabinSide_P,CabinSide_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,False,0,...,False,False,True,False,False,True,False,False,False,True
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,True,0,...,False,False,False,True,True,False,False,False,False,True
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,...,False,False,False,True,False,True,False,False,False,True
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,...,False,False,False,True,False,True,False,False,False,True
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,True,1,...,False,False,False,True,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,98,...,False,False,True,False,False,True,False,True,False,False
8689,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,1499,...,True,False,False,True,True,False,False,False,True,False
8690,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,1500,...,True,False,False,True,True,False,False,False,False,True
8691,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,608,...,False,False,False,True,False,True,False,True,False,False


# Training of the model

In [111]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

Here the process is reimplemented in single function

In [112]:
def preprocess(df):
  # split based on values
  df[["CabinDeck", "CabinNum", "CabinSide"]] = df["Cabin"].str.split("/", expand=True)
  df["CabinNum"] = df["CabinNum"].astype('Int64')

  df[['PassengerGroup', 'GroupMember']] = df['PassengerId'].str.split('_', expand=True)

  # fill with mean
  df['Age'] = df['Age'].fillna(df['Age'].mean())

  # fill with 0, since these are
  cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
  df[cols] = df[cols].fillna(0)

  # fill with -1, since 0 is also present in the data
  df['CabinNum'] = df['CabinNum'].fillna(-1)

  # drop columns that were splitted and change
  df.drop(columns=["Cabin", "PassengerId", "Name"], axis=1, inplace=True)

  # these are basically bollean flags, since not present is less common it will be filled as such
  df["CryoSleep"] = df["CryoSleep"].astype("Int64").fillna(0)
  df["VIP"] = df["VIP"].astype("Int64").fillna(0)

  df = pd.get_dummies(df, columns= ['CabinDeck', 'CabinSide', 'HomePlanet', 'Destination'])

  return df


In [113]:
df = pd.read_csv(train_path)

In [114]:
df = preprocess(df)

In [115]:
X = df.drop("Transported", axis=1)
y = df["Transported"]

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [117]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

Cross validation of the data

In [118]:
rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

scores = cross_val_score(rf, X, y, cv=kf, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Mean CV accuracy:", scores.mean())


Cross-validation scores: [0.79528465 0.79988499 0.80563542 0.81012658 0.7986191 ]
Mean CV accuracy: 0.8019101490149161


In [119]:
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [120]:
df_test = pd.read_csv(test_path)

In [121]:
passenger_ids = df_test["PassengerId"]

In [122]:
df_test_input = preprocess(df_test)

In [123]:
y = rf.predict(df_test_input)

In [124]:
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Transported": y,
})

In [125]:
submission.to_csv("submission.csv", index=False)

**Export model for later learned feature inspection**

In [126]:
import joblib

joblib.dump(rf, "train_0001_spaceship_titanic_baseline_rf.pkl")

['train_0001_spaceship_titanic_baseline_rf.pkl']